<a href="https://colab.research.google.com/github/Atik-Ahamed/HackerEarth-Deep-Learning-cha...nge-Identify-the-dance-form/blob/master/HackerEarth_Deep_Learning_challenge_Identify_the_dance_form.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing files

In [ ]:
import os
import pandas as pd
import random
import cv2
from sklearn import metrics
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
import numpy as np
import torchvision.models as models
from torchsummary import summary
import math
from PIL import Image
from IPython.display import clear_output
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import torchvision

import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from tqdm import tnrange
import matplotlib as mpl
import matplotlib.cm as cm
from collections import defaultdict
import torch.nn.functional as F
import torch.nn as nn
from torch import Tensor
from torch.nn  import functional as F
import torchvision.transforms.functional as TF
from torch.autograd import Variable
from torchvision import models
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import torchvision.transforms as T
import torchvision.transforms.functional as F
import pandas as pd
from sklearn.utils import class_weight
from sklearn.metrics import f1_score
from math import pi
from math import cos
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# CONFIG

In [ ]:
model_name ='resnet50' #current model 
#model_name = 'unet'
pretrained =True 
W=640 #image width
H=480 #image height
split=.15
random_state=1000
PATH='/content/drive/My Drive/ml/HackerEarthDanceMove/models/' #this is where the models will be saved after training
SUB_PATH='/content/drive/My Drive/ml/HackerEarthDanceMove/subfiles/' #Here the submission files will be saved
EPOCH= 100
LR=1e-3 #Initial Learning Rate
batch_size = 1 
STEP_SIZE=30
c_w=False #Class weight to balance imbalenced data
cur_time_stamp=str(time.time()) #every time unique time is generated, hene the model and submission will be renamed according to this
saved_model_name=PATH+cur_time_stamp+'_'+model_name
saved_submission_file=SUB_PATH+cur_time_stamp+'_'+'sub.csv'
#snapshot ensembling settings
epochs_per_cycle=100 
cycles=5
SE=False

# EDA

In [ ]:
#os.listdir('/content/drive/My Drive/datasets')
"""Unzipping the downloaded dataset"""
!unzip -q '/content/drive/My Drive/ml/HackerEarthDanceMove/HackerEarthDanceMoveChallenge.zip'

In [ ]:
train_df=pd.read_csv('/content/dataset/train.csv')
test_df=pd.read_csv('/content/dataset/test.csv')
print(train_df.head())
print(train_df.describe())

In [ ]:
train_df['target'].value_counts().plot(kind='bar',color='grey')
plt.title('target class distribution')

In [ ]:
x_data=train_df['Image']
y_data=train_df['target']
y_data=np.array(y_data)
class_weights=None
if c_w==True:
  class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_data),
                                                 y_data)
  print('The class weights are')
  print(class_weights)  

le = LabelEncoder()
ohe = OneHotEncoder(sparse=False)
y_data = le.fit_transform(y_data)
y_data = ohe.fit_transform(y_data.reshape(-1,1))
#print(y_data.shape)
#print(y_data)

inv_y_data = ohe.inverse_transform(y_data)
inv_y_data = le.inverse_transform(inv_y_data.astype(int).ravel())
#print(inv_y_data)



#y_data=pd.get_dummies(y_data)
x_train,x_val,y_train,y_val=train_test_split(x_data,y_data,test_size=split,random_state=random_state)

y_train,y_val=np.array(y_train),np.array(y_val)

print(x_train.shape,y_train.shape,x_val.shape,y_val.shape)

# Datagenerator

In [ ]:
class Generate_data(Dataset):
    def __init__(self,length,data_type,transform=None):

        self.transform = transform
        self.length=length
        self.data_type=data_type

    def __len__(self):
        return self.length


    def __getitem__(self, idx):
        if self.data_type=='train':
          image=Image.open('/content/dataset/train/'+x_train.iloc[idx])    
          if self.transform:
            image=self.transform(image)
          #print(image.shape)
          label=y_train[idx]
          return [image,label]
        
        if self.data_type=='val':
          image=Image.open('/content/dataset/train/'+x_val.iloc[idx])    
          if self.transform:
            image=self.transform(image)
          label=y_val[idx]
          return [image,label]
        
        if self.data_type=='test':
          image=Image.open('/content/dataset/test/'+test_df.iloc[idx]['Image'])    
          if self.transform:
            image=self.transform(image)      
          return image


trans_train = transforms.Compose([
                                  
    transforms.Resize((W,H)),

    
    #Play with some augmentations check what what works better
    
    #torchvision.transforms.RandomChoice([
    #torchvision.transforms.ColorJitter(brightness=10, contrast=10),
    #transforms.RandomRotation(20),
    #torchvision.transforms.RandomHorizontalFlip(),
    #torchvision.transforms.RandomCrop(2),
    #torchvision.transforms.RandomVerticalFlip(),
    #torchvision.transforms.RandomAffine((-5,5)),
    #]), 
   # ImageNetPolicy(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) #imagenet normalization policy check if it works better or not
])

trans_val = transforms.Compose([
    transforms.Resize((W,H)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_set = Generate_data(length=x_train.shape[0],data_type='train', transform = trans_train)
val_set = Generate_data(length=x_val.shape[0],data_type='val', transform = trans_val)
test_set = Generate_data(length=test_df.shape[0],data_type='test',transform=trans_val)
image_datasets = {
    'train': train_set, 'val': val_set, 'test':test_set
}



dataloader = {
    'train': DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0),
    'val': DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=0),
    'test': DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0),
}

In [ ]:
inputs,label=next(iter(dataloader['train']))
def reverse_transform(inp):
    inp = inp.numpy().transpose((1, 2, 0))
    #print(inp.shape)
    if pretrained == True:
      mean = np.array([0.485, 0.456, 0.406])
      std = np.array([0.229, 0.224, 0.225])
      inp = std * inp + mean
    #inp = np.clip(inp, 0, 1)
    
    inp = (inp * 255).astype(np.uint8)
    #inp=np.reshape(inp,(inp.shape[0],inp.shape[1]))

    return inp

#print(inputs.shape)
plt.imshow(reverse_transform(inputs[0]))

label_data = ohe.inverse_transform(label)
label_data = le.inverse_transform(label_data.astype(int).ravel())
plt.title(label_data[0])

# UNet

In [ ]:
from collections import OrderedDict

import torch
import torch.nn as nn


class UNet(nn.Module):

    def __init__(self, in_channels=3, out_channels=8, init_features=32):
        super(UNet, self).__init__()

        features = init_features
        self.encoder1 = UNet._block(in_channels, features, name="enc1")
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = UNet._block(features, features * 2, name="enc2")
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = UNet._block(features * 2, features * 4, name="enc3")
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = UNet._block(features * 4, features * 8, name="enc4")
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.bottleneck = UNet._block(features * 8, features * 16, name="bottleneck")

        self.upconv4 = nn.ConvTranspose2d(
            features * 16, features * 8, kernel_size=2, stride=2
        )
        self.decoder4 = UNet._block((features * 8) * 2, features * 8, name="dec4")
        self.upconv3 = nn.ConvTranspose2d(
            features * 8, features * 4, kernel_size=2, stride=2
        )
        self.decoder3 = UNet._block((features * 4) * 2, features * 4, name="dec3")
        self.upconv2 = nn.ConvTranspose2d(
            features * 4, features * 2, kernel_size=2, stride=2
        )
        self.decoder2 = UNet._block((features * 2) * 2, features * 2, name="dec2")
        self.upconv1 = nn.ConvTranspose2d(
            features * 2, features, kernel_size=2, stride=2
        )
        self.decoder1 = UNet._block(features * 2, features, name="dec1")

        self.conv = nn.Conv2d(
            in_channels=features, out_channels=out_channels, kernel_size=1
        )
        self.sig=torch.nn.Sigmoid()
        self.fc=nn.Linear(2457600,8)

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))

        bottleneck = self.bottleneck(self.pool4(enc4))

        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)
        conv1= self.conv(dec1)
        conv1=conv1.view(-1,conv1.shape[1]*conv1.shape[2]*conv1.shape[3])
        out=self.fc(conv1)
        #print(conv1.shape)
        #out=self.sig(conv1)

        
        return out

    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (
                        name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (
                        name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                ]
            )
        )


# Getting The Model

In [ ]:
def get_model(model_name):
    
  if model_name=='resnet50':
    if pretrained==True:
      model_ft = models.resnet50(pretrained=True)
    else:
      model_ft = models.resnet50(pretrained=False)
  if model_name=='unet':
    model=UNet()
    if use_cuda:
      model=model.cuda()
    return model

  #model_ft = models.resnet50(pretrained=False)
  #model_ft=models.resnet152(pretrained=True)
  #model_ft=models.resnet18(pretrained=True)
  #print(summary(model_ft, input_size=(3, 256, 256)))
  num_ftrs = model_ft.fc.in_features
  #print(num_ftrs)

  model_ft.fc = nn.Linear(num_ftrs, y_train.shape[1])
  model=model_ft
  if(use_cuda):
    model=model.cuda()
  del model_ft
  #print(summary(model, input_size=(3, W, H)))
  return model

In [ ]:
torch.cuda.empty_cache()
model=get_model(model_name)

In [ ]:
# model = pretrainedmodels.__dict__[model_name](num_classes=8,pretrained=)

In [ ]:
print(summary(model,(3,640,480)))

# Performance Metrics

In [ ]:
def calc_loss(preds,labels,metrics,class_weights=None):
    #class_weights=torch.from_numpy(class_weights)
    if c_w==True:
      if use_cuda:
        class_weights=torch.cuda.FloatTensor(class_weights)
      ce_loss=torch.nn.CrossEntropyLoss(weight=class_weights)
    else:
      ce_loss=torch.nn.CrossEntropyLoss()
    labels=torch.from_numpy(np.argmax(labels.data.cpu().numpy(),axis=1)).cuda()
    
    loss=ce_loss(preds,labels)
    metrics['loss'] += loss.data.cpu().numpy() * labels.size(0)
    return loss

def calc_acc(preds,labels,metrics):
    _, preds = torch.max(preds, 1)#pred contains the max valued index
    labels=torch.from_numpy(np.argmax(labels.data.cpu().numpy(),axis=1)).cuda()
    #print(preds.shape,labels.shape)
    metrics['Acc.'] += torch.sum(preds == labels.data)

def calc_f1_score(preds,labels,metrics):
    _, preds = torch.max(preds, 1)#pred contains the max valued index
    preds=preds.data.cpu().numpy()
    labels=np.argmax(labels.data.cpu().numpy(),axis=1)
    #print(preds.shape,labels.shape)
    f1=f1_score(labels,preds,average = 'macro')
    f1=f1* labels.shape[0]
    metrics['F1_Score'] +=f1

    
def print_metrics(metrics, epoch_samples, phase):
    outputs = []
    for k in metrics.keys():
        outputs.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))

    print("{}: {}".format(phase, ", ".join(outputs)))




# Train Functionality

In [ ]:
def train_model(optimizer, scheduler, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10
    best_acc=-1
    best_f1 = -1

    for epoch in tnrange(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('~' * 10)

        since = time.time()

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                for param_group in optimizer.param_groups:
                    print("LR", param_group['lr'])

                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            metrics = defaultdict(float)
            epoch_samples = 0

            for inputs, labels in dataloader[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = calc_loss(outputs, labels, metrics,class_weights)
                    calc_acc(outputs, labels,metrics)#no return just calc Acc.
                    calc_f1_score(outputs,labels,metrics)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                epoch_samples += inputs.size(0) #number of total samples in one epoch

            #after each epoch ends the following lines prints and measures the metrics
            print_metrics(metrics, epoch_samples, phase)
            epoch_loss = metrics['loss'] / epoch_samples
            epoch_acc = metrics['Acc.'] / epoch_samples
            epoch_f1 = metrics['F1_Score'] / epoch_samples
            
            # deep copy the model
            if phase == 'val':
              if epoch_loss<best_loss:
                best_model_wts = copy.deepcopy(model.state_dict())
                best_loss=epoch_loss
              if epoch_acc>best_acc:
                best_acc=epoch_acc
              if epoch_f1>best_f1:
                best_f1=epoch_f1               
                

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        #print('val loss: ',epoch_loss,' val acc: ',epoch_acc,' val f1 ',epoch_f1)


    print('Best val loss: {:4f}'.format(best_loss))
    print('Best val acc: {:4f}'.format(best_acc))
    print('Best val f1: {:4f}'.format(best_f1))
   
     # load best model weights
    model.load_state_dict(best_model_wts)
    return model


# Training and validation Specific model

In [ ]:
"""
Model is trained with adam optimizer and saved everytime if current epoch loss is less than the untill observed best validation loss
"""
if SE==False:
  optimizer_ft = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=LR)

  exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=STEP_SIZE, gamma=0.1)

  model = train_model(optimizer_ft, exp_lr_scheduler, num_epochs=EPOCH)

# Snapshot ensembling

## Functionality for training SE

In [ ]:
def proposed_lr(initial_lr, iteration, epoch_per_cycle):
    # proposed learning late function
    return initial_lr * (cos(pi * iteration / epoch_per_cycle) + 1) / 2

def train_se(epochs_per_cycle, cycles, initial_lr, train_loader, vis=None):

    snapshots = []
    _lr_list, _loss_list = [], []
    count = 0
    #epochs_per_cycle = epochs // cycles
    optimizer = optim.SGD(model.parameters(), lr=initial_lr)

    for i in range(cycles):


        best_model_wts = copy.deepcopy(model.state_dict())
        best_loss = 1e10
        best_acc=-1
        best_f1 = -1
        for j in range(epochs_per_cycle):
            lr = proposed_lr(initial_lr, j, epochs_per_cycle)
            for param_group in optimizer.param_groups:
              param_group['lr']=lr
            #optimizer.state_dict()["param_groups"][0]["lr"] = lr



            print('Cycle {} Epoch {}/{}'.format(i+1,j+1, epochs_per_cycle))
            print('~' * 10)

            since = time.time()

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    for param_group in optimizer.param_groups:
                        print("LR", param_group['lr'])

                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                metrics = defaultdict(float)
                epoch_samples = 0

                for inputs, labels in dataloader[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    
                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        loss = calc_loss(outputs, labels, metrics,class_weights)
                        calc_acc(outputs, labels,metrics)#no return just calc Acc.
                        calc_f1_score(outputs,labels,metrics)

                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    epoch_samples += inputs.size(0) #number of total samples in one epoch

                    #after each epoch ends the following lines prints and measures the metrics
                print_metrics(metrics, epoch_samples, phase)
                epoch_loss = metrics['loss'] / epoch_samples
                epoch_acc = metrics['Acc.'] / epoch_samples
                epoch_f1 = metrics['F1_Score'] / epoch_samples
                    
                # deep copy the model
                if phase == 'val':
                  if epoch_loss<best_loss:
                    best_loss=epoch_loss
                    best_model_wts = copy.deepcopy(model.state_dict())
                  if epoch_acc>best_acc:
                    best_acc=epoch_acc
                  if epoch_f1>best_f1:
                    best_f1=epoch_f1               
                       

            time_elapsed = time.time() - since
            print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
            #print('val loss: ',epoch_loss,' val acc: ',epoch_acc,' val f1 ',epoch_f1)

        print('Cycle: {}'.format(i+1))
        
        print('Best val loss: {:4f}'.format(best_loss))
        print('Best val acc: {:4f}'.format(best_acc))
        print('Best val f1: {:4f}'.format(best_f1)) 
        print('*'*50)       
        # load best model weights
        model.load_state_dict(best_model_wts)
        snapshots.append(model.state_dict())
    return snapshots


## Training SE

In [ ]:
snapshots=None
if SE==True:
  snapshots=train_se(epochs_per_cycle, cycles, LR, dataloader, vis=None)

# Functionality for testing Snapshot ensembling

In [ ]:
def get_se_pred(inputs,snapshots):
    torch.cuda.empty_cache()
    outputs=[]
    for data in dataloader['test']:
      for m in snapshots:
        model.load_state_dict(m)
        model.eval()
        outputs.append(model(inputs))
        torch.cuda.empty_cache()
        print('done')
    yhats = array(outputs)
    res=np.sum(yhats,axis=0)

    return res

# Testing on Test Set

In [ ]:
def get_test_results(snapshots=None):
  y_test=np.empty(0)
  if snapshots is None:
    model.eval()
  for inputs in dataloader['test']:
    inputs = inputs.to(device)
    if snapshots is not None:
      outputs=get_se_pred(inputs,snapshots)
    else:
      outputs = model(inputs)
    softmax=torch.nn.Softmax();
    outputs=softmax(outputs)
    outputs=outputs.data.cpu().numpy()
    outputs=np.argmax(outputs,axis=1)

    outputs = le.inverse_transform(outputs.astype(int).ravel())
    y_test=np.append(y_test,outputs)


  return y_test



In [ ]:
y_test=get_test_results(snapshots)

# Saving and loading the model

In [ ]:
if SE==False:
  torch.save(model.state_dict(), saved_model_name)

# .CSV submissions ensemble

In [ ]:
"""
This is for top scoring csv file ensembles by Hard voting method.

"""
csvs=os.listdir('/content/drive/My Drive/ml/HackerEarthDanceMove/submissions')
classes=['manipuri','bharatanatyam','odissi','kathakali','kathak','sattriya','kuchipudi','mohiniyattam']
if len(csvs)>1:   
    y_test=np.zeros(shape=(test_df.shape[0],y_train.shape[1]))


    for sub in csvs:
      cur_sub=pd.read_csv('/content/drive/My Drive/ml/HackerEarthDanceMove/submissions/'+sub)
      cur_sub=cur_sub['target']
      cur_sub=le.fit_transform(cur_sub)
      cur_sub=ohe.fit_transform(cur_sub.reshape(-1,1))
      # print(cur_sub.shape)
      # print(cur_sub)
      # break
      for i,x in enumerate(cur_sub):
        y_test[i]=y_test[i]+x
        # print(y_test[i])
      #   break
      # break

    #print(freq)
    y_test=np.argmax(y_test,axis=1)
    y_test = le.inverse_transform(y_test.astype(int).ravel())


# Making submission files

In [ ]:
submission = pd.DataFrame( columns = ['Image','target'])
submission['Image']=test_df['Image']
submission["target"]=y_test
print(submission.head())
submission.to_csv(saved_submission_file, index=False)